In [2]:
""" İşlenmiş veri setini yükleme ve kontrol """
import polars as pl
from pathlib import Path

# optimizasyon sonrası parquet verinin yolu
PROCESSED_DATA_PATH = Path("../data/processed/metropt3_optimized.parquet")

# Parqueti oku
df = pl.read_parquet(PROCESSED_DATA_PATH)

# Temel bilgiler
print(f"Satır sayısı : {df.shape[0]:,}")
print(f"Sütun sayısı : {df.shape[1]}")
print(f"Başlangıç   : {df['timestamp'].min()}")
print(f"Bitiş       : {df['timestamp'].max()}")
print(f"Süre        : {df['timestamp'].max() - df['timestamp'].min()}")

Satır sayısı : 1,516,948
Sütun sayısı : 16
Başlangıç   : 2020-02-01 00:00:00
Bitiş       : 2020-09-01 03:59:50
Süre        : 213 days, 3:59:50


Parqueti rame tam yükledik. artık lazy moda gerek yok baya küçülttük saten. Veriler temiz 1.5 milyon satır, 7 aylık veri, 16 sensör.

In [3]:
""" Zaman aralığı kontrolü """
# Timestamp farkları
time_diffs = df['timestamp'].diff().drop_nulls()
#.diff ardışık zaman damgası farkını hesaplar. .drop_nulls() ilk kayıtta oluşan null değeri temizler.

print("Zaman aralığı istatistikleri:")
print(f"Min  : {time_diffs.min()}") 
print(f"Max  : {time_diffs.max()}") 
print(f"Ortalama : {time_diffs.mean()}")
print(f"En sık görülen aralık : {time_diffs.mode()[0]}")

# Beklenmedik büyük boşluklar — 60 saniyeden fazla atlayan kayıtlar
gaps = df.filter(time_diffs.append(pl.Series([None])).shift(1) > pl.duration(seconds=60))
print(f"\n60 saniyeden uzun boşluk sayısı: {len(gaps)}")

Zaman aralığı istatistikleri:
Min  : 0:00:08
Max  : 2 days, 0:01:58
Ortalama : 0:00:12.141221
En sık görülen aralık : 0:00:10

60 saniyeden uzun boşluk sayısı: 331


Ortalama 10sn de bir veri akışı görmemiz lazım ancak 331 tane incelememiz gereken ciddi kopmalar var en önemlisi 2 günlük en büyük kopma var. Mantıksal yürütmeler ve tüm sensörlerdeki durumların o anlardaki halleri ile karşılaştırmalar sonucunda genel bakım mı, teknik onarım mı, birimsel arıza mı tespitleri yapıcaz

In [4]:
# Boşlukları büyükten küçüğe sırala ve göster
time_diffs_series = df['timestamp'].diff()

gaps_df = df.with_columns(time_diffs_series.alias('gap') #timestamp farklarını gap sütunu olarak ekle
).filter(pl.col('gap') > pl.duration(seconds=60) # gap değeri 60 saniyeden büyük olanları filtrele
).select(['timestamp', 'gap'] # sadece timestamp ve gap sütunlarını seç
).sort('gap', descending=True) # gap sütununa göre büyükten küçüğe sırala uzun kesintiler üstte

print(gaps_df.head(20))

shape: (20, 2)
┌─────────────────────┬────────────────┐
│ timestamp           ┆ gap            │
│ ---                 ┆ ---            │
│ datetime[μs]        ┆ duration[μs]   │
╞═════════════════════╪════════════════╡
│ 2020-04-27 01:12:49 ┆ 2d 1m 58s      │
│ 2020-06-28 23:07:43 ┆ 1d 12h 14m 36s │
│ 2020-03-01 04:00:09 ┆ 1d 4h 3m 1s    │
│ 2020-08-05 08:23:01 ┆ 1d 40m 33s     │
│ 2020-05-25 01:14:14 ┆ 1d 34m 51s     │
│ …                   ┆ …              │
│ 2020-03-30 07:33:30 ┆ 11h 44m 16s    │
│ 2020-03-20 06:25:41 ┆ 9h 18m 55s     │
│ 2020-05-01 21:26:05 ┆ 8h 47m 39s     │
│ 2020-06-02 18:47:39 ┆ 8h 47m 34s     │
│ 2020-04-30 23:34:46 ┆ 8h 28m 46s     │
└─────────────────────┴────────────────┘


In [5]:
""" Oturum tespiti — 60 saniyeden uzun boşluklar ayrı ayrı bölme """
session_gap = pl.duration(seconds=60) # Oturum boşluk eşiği (ayraç)

df = df.with_columns([
    pl.col('timestamp').diff().alias('time_diff') #timestamp farklarını time_diff sütunu olarak ekle
]).with_columns([
    # Boşluk > 60sn ise yeni oturum başlıyor — kümülatif toplam ile oturum ID üret
    (pl.col('time_diff') > session_gap)
    .cast(pl.Int32) #.cast(pl.Int32) ile boolean değeri 0 ve 1'e dönüştürür. True (boşluk > 60sn) için 1, False için 0 olur.
    .cum_sum() # .cum_sum() ile 0 ve 1'lerin kümülatif toplamını alır. Böylece herboşluk > 60 sn olduğunda oturum ID'si artar.
    .alias('session_id') # session_id sütunu olarak adlandır
])

# Oturum istatistikleri
session_stats = df.group_by('session_id').agg([
    pl.col('timestamp').min().alias('session_start'), # Oturumun başlangıç zamanı
    pl.col('timestamp').max().alias('session_end'), # Oturumun bitiş zamanı
    pl.col('timestamp').count().alias('record_count') # Her oturumdaki kayıt sayısı
]).with_columns([
    (pl.col('session_end') - pl.col('session_start')).alias('session_duration')
]).sort('session_start') # Oturum başlangıcına göre sırala

print(f"Toplam oturum sayısı : {session_stats.shape[0]}")
print(f"\nİlk 10 oturum:")
print(session_stats.head(10))

Toplam oturum sayısı : 333

İlk 10 oturum:
shape: (10, 5)
┌────────────┬─────────────────────┬─────────────────────┬──────────────┬──────────────────┐
│ session_id ┆ session_start       ┆ session_end         ┆ record_count ┆ session_duration │
│ ---        ┆ ---                 ┆ ---                 ┆ ---          ┆ ---              │
│ i32        ┆ datetime[μs]        ┆ datetime[μs]        ┆ u32          ┆ duration[μs]     │
╞════════════╪═════════════════════╪═════════════════════╪══════════════╪══════════════════╡
│ null       ┆ 2020-02-01 00:00:00 ┆ 2020-02-01 00:00:00 ┆ 1            ┆ 0µs              │
│ 0          ┆ 2020-02-01 00:00:10 ┆ 2020-02-01 12:48:40 ┆ 4653         ┆ 12h 48m 30s      │
│ 1          ┆ 2020-02-01 12:53:47 ┆ 2020-02-01 19:40:04 ┆ 2460         ┆ 6h 46m 17s       │
│ 2          ┆ 2020-02-01 23:15:33 ┆ 2020-02-01 23:20:21 ┆ 30           ┆ 4m 48s           │
│ 3          ┆ 2020-02-02 04:34:27 ┆ 2020-02-02 07:37:40 ┆ 1110         ┆ 3h 3m 13s        │
│ 4         

Şüpheli 331 kesintiyi baz alarak öncesi ve sonrasını oturumlara böldük. toplamda 332 oturumun kesinti başlangıç - bitiş - oturum içinde kaç kere veri girildiği (yani ortalama 10sn almıştık her veri girişini oturumda kaç kere her 10sn içinde veri girişi olduğudur) - oturumun toplam süresi.

Burada olay .cum_sum metodudur. cast ile 60snden küçük-büyük olarak binary yaptığımız değerleri kümülatif olarak toplar yani 0 + 0 + 0 diye giderken 1 gelince toplam değişir bu değişim yeni oturum sinyalidir. sonra tekrar 0 lar ile toplanınca değişim yok tekrardan 1 gelmesi ile değişince yine yeni oturum sinyali oluşur.

Genel olarak endüstride makine arızasından 1-4 saat öncesinde sensörlere yansıması oluyormuş bundan dolayı biz ortalama 2 saat alarak her oturumun 2 saat öncesini pencereleyerek inceleyeceğiz.

In [6]:
""" En az 3 saatlik oturumları filtreleme """
min_duration = pl.duration(hours=3) # 3 saat olan oturumları filtrele

valid_sessions = session_stats.filter( # boş olmayan ve eşiği geçen oturumları filtrele
    (pl.col('session_id').is_not_null()) &
    (pl.col('session_duration') >= min_duration)
)

print(f"Toplam oturum        : {session_stats.shape[0]}")
print(f"Geçerli oturum       : {valid_sessions.shape[0]}")
print(f"Filtrelenen oturum   : {session_stats.shape[0] - valid_sessions.shape[0]}")
print(f"\nGeçerli oturumların özeti:")
print(valid_sessions.describe()) # geçerli oturumların istatistiksel özetini göster (count, mean, max, min, std, vb.)

Toplam oturum        : 333
Geçerli oturum       : 183
Filtrelenen oturum   : 150

Geçerli oturumların özeti:
shape: (9, 6)
┌────────────┬────────────┬───────────────────┬──────────────────┬──────────────┬──────────────────┐
│ statistic  ┆ session_id ┆ session_start     ┆ session_end      ┆ record_count ┆ session_duration │
│ ---        ┆ ---        ┆ ---               ┆ ---              ┆ ---          ┆ ---              │
│ str        ┆ f64        ┆ str               ┆ str              ┆ f64          ┆ str              │
╞════════════╪════════════╪═══════════════════╪══════════════════╪══════════════╪══════════════════╡
│ count      ┆ 183.0      ┆ 183               ┆ 183              ┆ 183.0        ┆ 183              │
│ null_count ┆ 0.0        ┆ 0                 ┆ 0                ┆ 0.0          ┆ 0                │
│ mean       ┆ 168.464481 ┆ 2020-05-22        ┆ 2020-05-23       ┆ 8113.912568  ┆ 22:30:14.377049  │
│            ┆            ┆ 06:18:51.163934   ┆ 04:49:05.540983  ┆   

En az 3 saatlik oturumları filtreleme sebebimiz endüstriyel olarak 2 saatlik pencere yani inceleme alanı belirledik ancak 1 saat de makinenin düzgün çalışıyor mu kontrolü için referans aralığı bırakılmalı bundan dolayı sağlıklı analiz tespiti için oturumlar min 3 saat olmaldır.

183 tane oturum tespit ettik. oturum başına ortalama 22 saat ölçüm süresi, 19 saat de veri girişi var bu değerler tespitler için yeterli olacak.

In [7]:
""" 2 SAATLİK ŞÜPHELİ PENCERE EKLEME """
# Her geçerli oturumun son 2 saatini şüpheli pencere olarak işaretle
suspect_window = pl.duration(hours=2)

df = df.with_columns([
    pl.lit(0).alias('is_suspect')  # Önce hepsini 0 yap
])

# Her geçerli oturum için son 2 saati işaretle
for row in valid_sessions.iter_rows(named=True): # valid_sessions içindeki her satırı row olarak alır. named=True ile sütun adlarıyla erişim sağlar
    session_end = row['session_end'] # Oturumun bitiş zamanı
    suspect_start = session_end - suspect_window # Şüpheli pencerenin başlangıç zamanı (oturum sonundan 2 saat öncesi)
    
    df = df.with_columns([ # Şüpheli pencere içinde olan kayıtları is_suspect = 1 yap
        pl.when(
            (pl.col('session_id') == row['session_id']) & # Aynı oturumda olan ve
            (pl.col('timestamp') >= suspect_start) # Şüpheli pencere başlangıcından sonra olan
        ) 
        .then(1) # Bu koşulları sağlayan kayıtları 1 yap
        .otherwise(pl.col('is_suspect')) # Diğer kayıtlar is_suspect değerini değiştirme
        .alias('is_suspect') # is_suspect sütununu güncelle
    ])

# Sonucu kontrol et
print(f"Toplam kayıt        : {df.shape[0]:,}")
print(f"Normal kayıt        : {df.filter(pl.col('is_suspect') == 0).shape[0]:,}")
print(f"Şüpheli kayıt       : {df.filter(pl.col('is_suspect') == 1).shape[0]:,}")
print(f"Şüpheli oran        : %{df['is_suspect'].mean() * 100:.1f}")

Toplam kayıt        : 1,516,948
Normal kayıt        : 1,384,825
Şüpheli kayıt       : 132,123
Şüpheli oran        : %8.7


ayırdığımız oturumların bitişinden önceki 2 saatlik zaman dilimini şüpheli pencereye aldık ve bu kısımlara özel kolon oluşturup "1" olarak kodladık. Burada +1 saatlik anormallik öncesi zamna olarak aldığımız kısmı es geçtik o kısımlar kolonda "0" olarak bırakarak ayırdık

132k şüpheli kayıt elde ettik. bu anomali tespiti yapmak için ideal miktardır.